# Run without recalibration

Will take a long time to recalibrate.

In [ ]:
import os

import matplotlib.pyplot as pl
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d
from fair import FAIR
from fair.interface import fill, initialise
from fair.io import read_properties

In [ ]:
scenarios = ['Baseline', 'MTFR', 'MTFR-Low']

In [ ]:
df_solar = pd.read_csv('../data_input/solar_erf_1750-2300_timebounds.csv')
df_volcanic = pd.read_csv('../data_input/volcanic_erf_1750-2300_timebounds.csv')

In [ ]:
df_configs = pd.read_csv('../data_input/calibrated_constrained_parameters_1.1.0.csv')
df_historical = pd.read_csv('../data_input/rcmip-emissions-historical-corrected-nox.csv', index_col='Variable')
df_ssp = pd.read_csv('../data_input/rcmip-emissions-reduced-future.csv')

In [ ]:
df_baseline = pd.read_csv('../data_input/gains-baseline.csv', index_col=0)
df_mtfr = pd.read_csv('../data_input/gains-mtfr.csv', index_col=0)
df_mtfr_low = pd.read_csv('../data_input/gains-mtfr-low.csv', index_col=0)

In [ ]:
valid_all = df_configs.index

In [ ]:
trend_shape = np.ones(551)
trend_shape[:271] = np.linspace(0, 1, 271)

In [ ]:
f = FAIR(ch4_method="Thornhill2021")
f.define_time(1750, 2051, 1)
f.define_scenarios(scenarios)
f.define_configs(valid_all)
species, properties = read_properties()
f.define_species(species, properties)
f.allocate()

In [ ]:
df_historical.index

In [ ]:
gains_species = ['CO2 FFI', 'CH4', 'NOx', 'VOC', 'CO']

In [ ]:
for specie in df_historical.index:
    fill(
        f.emissions, 
        df_historical.loc[specie, :].values.squeeze()[:, None, None], 
        timepoints=np.arange(1750.5, 2015), 
        specie=specie,
    )

In [ ]:
#df_ssp

In [ ]:
nongains_from = {
    "Baseline": "ssp245",
    "MTFR": "ssp245",
    "MTFR-Low": "ssp245"  # for fair comparison
}

In [ ]:
df_ssp.loc[(df_ssp['Scenario']=='ssp126') & (df_ssp['Variable']=='BC'), '2015':'2050'].values.squeeze()

In [ ]:
for scenario in scenarios:
    for specie in df_historical.index:
        if specie in gains_species:
            continue
        interpolator = interp1d(
            [2015.5, 2020.5, 2030.5, 2040.5, 2050.5], 
            df_ssp.loc[
                (df_ssp['Scenario']==nongains_from[scenario]) & (df_ssp['Variable']==specie),
                '2015':'2050'
            ].values.squeeze()
        )
        values = interpolator(np.arange(2015.5, 2051))
        fill(
            f.emissions, 
            values[:, None], 
            timepoints=np.arange(2015.5, 2051), 
            specie=specie,
            scenario=scenario
        )

In [ ]:
df_baseline

In [ ]:
for specie in gains_species:
    interpolator = interp1d(
        np.arange(1990.5, 2051, 5), 
        df_baseline.loc[specie, :].values.squeeze()
    )
    values = interpolator(np.arange(1990.5, 2051))
    fill(
        f.emissions, 
        values[:, None], 
        timepoints=np.arange(1990.5, 2051), 
        specie=specie,
        scenario="Baseline"
    )
    interpolator = interp1d(
        np.arange(1990.5, 2051, 5), 
        df_mtfr.loc[specie, :].values.squeeze()
    )
    values = interpolator(np.arange(1990.5, 2051))
    fill(
        f.emissions, 
        values[:, None], 
        timepoints=np.arange(1990.5, 2051), 
        specie=specie,
        scenario="MTFR"
    )
    interpolator = interp1d(
        np.arange(1990.5, 2051, 5), 
        df_mtfr_low.loc[specie, :].values.squeeze()
    )
    values = interpolator(np.arange(1990.5, 2051))
    fill(
        f.emissions, 
        values[:, None], 
        timepoints=np.arange(1990.5, 2051), 
        specie=specie,
        scenario="MTFR-Low"
    )

In [ ]:
df_volcanic.erf[:301]

In [ ]:
fill(
    f.forcing,
    df_volcanic.erf[:302].values[:,None,None] * df_configs["scale Volcanic"].values.squeeze(),
    specie="Volcanic",
)

In [ ]:
fill(
    f.forcing, 
    df_solar.erf[:271].values[:,None,None] * df_configs["solar_amplitude"].values.squeeze()
    + trend_shape[:271,None,None] * df_configs["solar_trend"].values.squeeze(),
    specie="Solar",
    timebounds=np.arange(1750, 2021)
)
fill(
    f.forcing,
    trend_shape[271:302,None,None] * df_configs["solar_trend"].values.squeeze(),
    specie="Solar",
    timebounds=np.arange(2021, 2052)
)

In [ ]:
# climate response
fill(f.climate_configs["ocean_heat_capacity"], df_configs.loc[:, "c1":"c3"].values)
fill(
    f.climate_configs["ocean_heat_transfer"],
    df_configs.loc[:, "kappa1":"kappa3"].values,
)
fill(f.climate_configs["deep_ocean_efficacy"], df_configs["epsilon"].values.squeeze())
fill(f.climate_configs["gamma_autocorrelation"], df_configs["gamma"].values.squeeze())
fill(f.climate_configs["sigma_eta"], df_configs["sigma_eta"].values.squeeze())
fill(f.climate_configs["sigma_xi"], df_configs["sigma_xi"].values.squeeze())
fill(f.climate_configs["seed"], df_configs["seed"])
fill(f.climate_configs["stochastic_run"], False)
fill(f.climate_configs["use_seed"], True)
fill(f.climate_configs["forcing_4co2"], df_configs["F_4xCO2"])

In [ ]:
# species level
f.fill_species_configs()

# carbon cycle
fill(f.species_configs["iirf_0"], df_configs["r0"].values.squeeze(), specie="CO2")
fill(
    f.species_configs["iirf_airborne"], df_configs["rA"].values.squeeze(), specie="CO2"
)
fill(f.species_configs["iirf_uptake"], df_configs["rU"].values.squeeze(), specie="CO2")
fill(
    f.species_configs["iirf_temperature"],
    df_configs["rT"].values.squeeze(),
    specie="CO2",
)

# aerosol indirect
fill(f.species_configs["aci_scale"], df_configs["beta"].values.squeeze())
fill(
    f.species_configs["aci_shape"],
    df_configs["shape Sulfur"].values.squeeze(),
    specie="Sulfur",
)
fill(
    f.species_configs["aci_shape"], df_configs["shape BC"].values.squeeze(), specie="BC"
)
fill(
    f.species_configs["aci_shape"], df_configs["shape OC"].values.squeeze(), specie="OC"
)

# methane lifetime baseline and sensitivity
fill(
    f.species_configs["unperturbed_lifetime"],
    9.99789416859927,
    specie="CH4",
)
fill(
    f.species_configs["ch4_lifetime_chemical_sensitivity"],
    0.000254099313466019,
    specie="CH4",
)
fill(
    f.species_configs["ch4_lifetime_chemical_sensitivity"],
    -0.000722664594950932,
    specie="N2O",
)
fill(
    f.species_configs["ch4_lifetime_chemical_sensitivity"],
    0.00161926353368172,
    specie="VOC",
)
fill(
    f.species_configs["ch4_lifetime_chemical_sensitivity"],
    -0.00251106743984572,
    specie="NOx",
)
fill(
    f.species_configs["ch4_lifetime_chemical_sensitivity"],
    -0.0000053273763975757,
    specie="Equivalent effective stratospheric chlorine",
)
fill(
    f.species_configs["lifetime_temperature_sensitivity"],
    -0.0408,
)

# emissions adjustments for N2O and CH4 (we don't want to make these defaults as people
# might wanna run pulse expts with these gases)
fill(f.species_configs["baseline_emissions"], 19.019783117809567, specie="CH4")
fill(f.species_configs["baseline_emissions"], 0.08602230754, specie="N2O")
fill(f.species_configs["baseline_emissions"], 19.423526730206152, specie="NOx")

# aerosol direct
for specie in [
    "BC",
    "CH4",
    "N2O",
    "NH3",
    "NOx",
    "OC",
    "Sulfur",
    "VOC",
    "Equivalent effective stratospheric chlorine",
]:
    fill(
        f.species_configs["erfari_radiative_efficiency"],
        df_configs[f"ari {specie}"],
        specie=specie,
    )

# forcing scaling
for specie in [
    "CO2",
    "CH4",
    "N2O",
    "Stratospheric water vapour",
    "Contrails",
    "Light absorbing particles on snow and ice",
    "Land use",
]:
    fill(
        f.species_configs["forcing_scale"],
        df_configs[f"scale {specie}"].values.squeeze(),
        specie=specie,
    )

for specie in [
    "CFC-11",
    "CFC-12",
    "CFC-113",
    "CFC-114",
    "CFC-115",
    "HCFC-22",
    "HCFC-141b",
    "HCFC-142b",
    "CCl4",
    "CHCl3",
    "CH2Cl2",
    "CH3Cl",
    "CH3CCl3",
    "CH3Br",
    "Halon-1211",
    "Halon-1301",
    "Halon-2402",
    "CF4",
    "C2F6",
    "C3F8",
    "c-C4F8",
    "C4F10",
    "C5F12",
    "C6F14",
    "C7F16",
    "C8F18",
    "NF3",
    "SF6",
    "SO2F2",
    "HFC-125",
    "HFC-134a",
    "HFC-143a",
    "HFC-152a",
    "HFC-227ea",
    "HFC-23",
    "HFC-236fa",
    "HFC-245fa",
    "HFC-32",
    "HFC-365mfc",
    "HFC-4310mee",
]:
    fill(
        f.species_configs["forcing_scale"],
        df_configs["scale minorGHG"].values.squeeze(),
        specie=specie,
    )

# ozone
for specie in [
    "CH4",
    "N2O",
    "Equivalent effective stratospheric chlorine",
    "CO",
    "VOC",
    "NOx",
]:
    fill(
        f.species_configs["ozone_radiative_efficiency"],
        df_configs[f"o3 {specie}"],
        specie=specie,
    )

# tune down volcanic efficacy
fill(f.species_configs["forcing_efficacy"], 0.6, specie="Volcanic")


# initial condition of CO2 concentration (but not baseline for forcing calculations)
fill(
    f.species_configs["baseline_concentration"],
    df_configs["co2_concentration_1750"].values.squeeze(),
    specie="CO2",
)

In [ ]:
# initial conditions
initialise(f.concentration, f.species_configs["baseline_concentration"])
initialise(f.forcing, 0)
initialise(f.temperature, 0)
initialise(f.cumulative_emissions, 0)
initialise(f.airborne_emissions, 0)

In [ ]:
f.run()

In [ ]:
pl.plot(f.concentration.loc[dict(specie='CH4', config=valid_all[0])])

In [ ]:
pl.plot(f.concentration.loc[dict(specie='CO2', config=valid_all[0])])

In [ ]:
pl.plot(f.alpha_lifetime.loc[dict(specie='CH4', config=valid_all[0])])

In [ ]:
pl.plot(f.emissions.loc[dict(specie='CH4', config=valid_all[0])])

In [ ]:
pl.plot(
    np.median(
        f.temperature.loc[
            dict(layer=0, scenario="Baseline")
        ] - f.temperature.loc[
            dict(layer=0, scenario="Baseline", timebounds=np.arange(1995, 2015))
        ].mean(axis=0) + 0.85,
        axis=1
    ),
)

pl.plot(
    np.median(
        f.temperature.loc[
            dict(layer=0, scenario="MTFR")
        ] - f.temperature.loc[
            dict(layer=0, scenario="MTFR", timebounds=np.arange(1995, 2015))
        ].mean(axis=0) + 0.85,
        axis=1
    ),
)

pl.plot(
    np.median(
        f.temperature.loc[
            dict(layer=0, scenario="MTFR-Low")
        ] - f.temperature.loc[
            dict(layer=0, scenario="MTFR-Low", timebounds=np.arange(1995, 2015))
        ].mean(axis=0) + 0.85,
        axis=1
    ),
)

In [ ]:
pl.plot(
    f.timebounds,
    np.median(
        f.temperature.loc[
            dict(layer=0, scenario="Baseline")
        ] - f.temperature.loc[
            dict(layer=0, scenario="Baseline", timebounds=np.arange(1995, 2015))
        ].mean(axis=0) + 0.85,
        axis=1
    ),
    label='Baseline'
)

pl.plot(
    f.timebounds,
    np.median(
        f.temperature.loc[
            dict(layer=0, scenario="MTFR")
        ] - f.temperature.loc[
            dict(layer=0, scenario="MTFR", timebounds=np.arange(1995, 2015))
        ].mean(axis=0) + 0.85,
        axis=1
    ),
    label='MTFR'
)

pl.plot(
    f.timebounds,
    np.median(
        f.temperature.loc[
            dict(layer=0, scenario="MTFR-Low")
        ] - f.temperature.loc[
            dict(layer=0, scenario="MTFR-Low", timebounds=np.arange(1995, 2015))
        ].mean(axis=0) + 0.85,
        axis=1
    ),
    label='MTFR-Low'
)
pl.xlim(2000, 2050)
pl.ylim(0.6, 2)
pl.grid()
pl.legend()